In [1]:
from dash import Dash, html, dcc, Input, Output, callback_context
import plotly.express as px
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
import dash_bootstrap_components as dbc

In [2]:
df = pd.read_feather("2025_KV_Lasse_data.feather").reset_index()
df.columns = df.columns.str.replace(".0", "")
df.head()

,index,DR0,DR1,DR3,DR4,DR5,DR6,DR7,DR8,DR9,...,tv2-kv25-danmark-16,tv2-kv25-danmark-17,tv2-kv25-danmark-18,tv2-kv25-danmark-19,tv2-kv25-danmark-20,parti,navn,job,alder,kreds
0,0,1.00,0.00,0.75,0.625,0.625,0.75,0.75,0.00,0.25,...,0.50,0.00,0.00,0.50,0.75,socialdemokratiet,Mark Søgaard,Ambulanceinstruktør,39,Assens
1,1,0.25,0.00,0.75,0.625,1.000,0.75,0.50,0.50,1.00,...,0.75,0.00,0.25,0.00,1.00,socialdemokratiet,Ivan Damsted,Pensonist,66,Assens
2,2,0.00,0.75,0.75,0.750,0.125,1.00,0.75,0.50,0.00,...,1.00,0.50,0.25,0.00,0.25,liberal alliance,Kenneth Ettrup Hansen,Selvstændig,30,Vejle
3,3,0.75,0.75,0.25,0.250,0.250,0.75,0.50,0.25,0.25,...,0.50,0.50,0.25,0.75,0.25,nærdemokraterne,Kitty Gamkinn,Pilot,46,Varde
4,4,0.25,0.75,0.25,0.750,0.375,1.00,0.75,0.25,0.00,...,0.25,0.75,1.00,0.75,0.25,det konservative folkeparti,Lars Storgaard,Borgmester,64,Favrskov


In [5]:
dk_spg = list(df.columns[df.columns.str.startswith("DR") | df.columns.str.startswith("tv2")])

In [6]:
color_dict = pd.read_json("various.json").apply(lambda x: x.str.lower()).set_index('bogstav_leg')['farver'].to_dict()


In [7]:
X = df[dk_spg]
y = df['parti']

lda = LinearDiscriminantAnalysis(n_components=2).fit(X, y)

q = pd.concat([
    df,
    pd.DataFrame(lda.transform(df[dk_spg]), columns=["X", "y"]).set_index(df.index)],
    #pd.DataFrame(PCA(n_components=2).fit_transform(X), columns=["X", "y"]).set_index(df.index)],
    axis=1)

In [53]:
bogfarve = pd.read_json("various.json").reset_index()
bogfarve['bogstav_leg'] = bogfarve['bogstav_leg'].str.lower()
bogfarve = bogfarve.set_index('bogstav_leg')
q['bogstav'] = q.parti.map(bogfarve['index']).fillna('X')

In [7]:
dr_sprg = pd.Series({
"DR0": "Uddannelse og skoler",
"DR1": "Ældrepleje og seniorservice",
"DR2": "Transport og mobilitet",
"DR3": "Grøn energi og klima",
"DR4": "Byudvikling og byggeri",
"DR5": "Religion og kulturel integration",
"DR6": "Kultur og fritid",
"DR7": "Sundhed og hospitaler",
"DR8": "Infrastruktur og anlægsprojekter",
"DR9": "Boliger og daginstitutioner",
"DR10": "Skatter og brugerbetaling",
"DR11": "Turisme og ferieområder",
"DR12": "Børnepasning og førskole",
"DR13": "Tryghed og sikkerhed",
"DR14": "Miljø og naturbeskyttelse",
"DR15": "Politik og kommunal styring",
"DR16": "Erhverv og virksomheder",
"DR17": "Social velfærd og udsatte grupper",
"DR18": "Sport og idrætsfaciliteter",
"DR19": "Børn og unge trivsel"
})

In [35]:
tv2_sprg = pd.read_json('TV2/tv2_sprg.json').set_index("id")['question']
#dr_sprgs = pd.read_json('raw_data/DR/questions.json')
#dr_sprgs.columns = dr_sprgs.columns.str.lower()
#dr_sprgs = pd.DataFrame()

sprgs = pd.concat([dr_sprg, tv2_sprg])
#sprgs.index = sprgs.index.astype('str')

#sprgs = sprgs.loc[dk_spg]
svar_muligheder = ['helt uenig','uenig','neutral','enig','helt enig']

In [36]:
def confidence_ellipse(x, y, n_std=1.96, size=100):
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    theta = np.linspace(0, 2 * np.pi, size)
    ellipse_coords = np.column_stack([ell_radius_x * np.cos(theta), ell_radius_y * np.sin(theta)])
    x_scale = np.sqrt(cov[0, 0]) * n_std
    x_mean = np.mean(x)
    y_scale = np.sqrt(cov[1, 1]) * n_std
    y_mean = np.mean(y)  
    translation_matrix = np.tile([x_mean, y_mean], (ellipse_coords.shape[0], 1))
    rotation_matrix = np.array([[np.cos(np.pi / 4), np.sin(np.pi / 4)], [-np.sin(np.pi / 4), np.cos(np.pi / 4)]])
    scale_matrix = np.array([[x_scale, 0], [0, y_scale]])
    ellipse_coords = ellipse_coords.dot(rotation_matrix).dot(scale_matrix) + translation_matrix
    
    path = f'M {ellipse_coords[0, 0]}, {ellipse_coords[0, 1]}'
    for k in range(1, len(ellipse_coords)):
        path += f'L{ellipse_coords[k, 0]}, {ellipse_coords[k, 1]}'
    path += ' Z'
    return path

In [37]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SOLAR, dbc.icons.BOOTSTRAP],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"},],
)

app.layout = dbc.Container([
	dcc.Store(id='bruger_coord'),
	dbc.Card(
		dbc.Container(
			[
				html.H2("Kommunalvalg 2025"),
				html.P("Analyse af hvor de enkelte kandidater står i forhold til hinanden og deres partier",
					   className="lead", ),
			],  # fluid=True,
		), body=True
	),
	dbc.Card(
		[
			dbc.Card([
                html.Label(["kreds filter:", dcc.Dropdown(id='kreds_valg', options=[{'value':'alle', 'label':'alle'}, *[{'value': x, 'label': x} for x in q.kreds.unique()]], value=['alle',], multi=True)]),
				dbc.Switch(id='parti_shadow', value=True, label="Tegn cirkler om partierne"),
				dbc.Switch(id='farveblind', value=False, label="Farveblind mode"),
			], body=True)
		],
	),

	dbc.Card(dcc.Graph(id='viz')),
	dbc.Card(html.P("(her kommer forudsigelser om hvilket parti en 'klikket' politiker burde være i)", id="svar_res")),
	dbc.Card([
		dcc.Markdown('''
		# SVAR
		### Tryk på politiker for at se deres svar eller svar selv for at se hvor DU ligger
		helt uenig  --  uenig  --  neutral  --  enig  --  helt enig
		'''),
		dbc.ListGroup([
			dbc.ListGroupItem([
				html.P(sprgs.loc[spg]),
				dcc.RadioItems(id=spg, options=[{'label': '', 'value': x / 4} for x in range(5)],
							   value=0, labelStyle={'display': 'inline-block'})
			]) for spg in dk_spg
		], flush=True)
	], body=True)
	, ],  # fluid=True
)

In [40]:
@app.callback(Output('viz', 'figure'), [Input('kreds_valg', 'value'),
	Input('parti_shadow', 'value'),
	Input('farveblind', 'value'),
	Input('bruger_coord', 'data')])
def update_graph(valgkreds_filter, shadow, farveblind, data):
    if 'alle' in valgkreds_filter:
        a = q
    else:
        a = q[q.kreds.isin(valgkreds_filter)]
        
    if farveblind:
        f1 = px.scatter(
            a, x='X', y='y', color='parti', color_discrete_map=color_dict, hover_data=['navn', 'job', 'alder'],
            custom_data=['index'], template="plotly_dark", labels={"X": "Hjalmesans", "y": "Fluplighed"},
            size='sized', text='bogstav', size_max=15
            # , width=1000  # , marginal_x='box'
        )
    else:
        f1 = px.scatter(
            a, x='X', y='y', color='parti', color_discrete_map=color_dict, hover_data=['navn', 'job', 'alder'],
            custom_data=['index'], template="plotly_dark", labels={"X": "Hjalmesans", "y": "Fluplighed"},
        )
        
    f1.layout.xaxis.fixedrange = True
    f1.layout.yaxis.fixedrange = True
    f1.update_layout(modebar_remove=['zoom', 'pan', 'select', 'lasso2d'])
    
    if shadow:
        for ii, (i, pari_data) in enumerate(q.groupby('parti')):
            if len(pari_data.X) > 100:
                f1.add_shape(type='path', path=confidence_ellipse(pari_data.X, pari_data.y), line_color='rgb(255,255,255,1)', fillcolor=color_dict[i], opacity=.4, )
    if data['dine_aktiv']:
        # f1.add_scatter(x=[data['dine_coords'][0]], y=[data['dine_coords'][1]], mode='markers', marker_symbol='star', marker_size=15)
        f1.add_vline(data['dine_coords'][0], line_dash="dash", line_color="pink")
        f1.add_hline(data['dine_coords'][1], line_dash="dash", line_color="pink")
    return f1

In [41]:
@app.callback([*[Output(x, 'value') for x in dk_spg], Output('svar_res', 'children'), Output('bruger_coord', 'data')],
			  {'clickData': Input('viz', 'clickData'), 'spg_in': [Input(x, 'value') for x in dk_spg]})
def display_click_data(clickData, spg_in):
	ctx = callback_context
	trigger_id = ctx.triggered[0]["prop_id"].split(".")[0]
	if trigger_id == 'viz':
		dine_aktiv = False
		if clickData and len(clickData['points']) != 0:
			idx = clickData['points'][0]['customdata'][0]
			navn = clickData['points'][0]['customdata'][1]
		else:
			idx = 1350
			navn = "(klik på nogen)"
		row = q[q['index'] == idx]
		parti = row['parti'].iloc[0]
		nyt_parti = lda.predict(row[dk_spg])[0]
		return [*[row[x].iloc[0] for x in dk_spg],
				f"Du har klikket på {navn}, {parti}. Vedkomne burde overveje {nyt_parti}",
				{'dine_aktiv': dine_aktiv, 'dine_coords': [0, 0]}]
	else:
		dine_aktiv = True
		a = pd.DataFrame(spg_in, index=dk_spg).T
		dine_coords = lda.transform(a)[0]
		return [*[x for x in spg_in],
				f"Dine koordinater er {dine_coords[0]:.1f}, {dine_coords[1]:.1f}. Du burde overveje {lda.predict(a)[0]}",
				{'dine_aktiv': dine_aktiv, 'dine_coords': dine_coords}]


In [42]:
#app.run_server(mode='jupyterlab')
#app.run_server(mode='external')
app.run(jupyter_mode="inline", jupyter_height=500, jupyter_width="100%")